# Clean - Clean Dataset
This example illustrate how to perform meta-blocking on a clean-clean dataset (data linkage), so when we have two datasets that do not contain duplicates and we want to discover the duplicates between them. 

In [123]:
import sparker
import random
import pandas as pd

In [124]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

## Load the data
sparkER provides wrappers to load CSV and JSON files.

First, we load the first dataset, and we extract the maximum id. The profiles ids of the second dataset will be assigned starting from this one.

*real_id_field* is the field that contains the identifier of the record.

In [125]:
# Profiles contained in the first dataset
profiles1 = sparker.CSVWrapper.load_profiles(file_path = 'file:///home/aditi/BigData/AAI_Project/CODE/Amazon.csv',
                                             start_id_from = 0,
                                             separator=",", header=True,
                                             real_id_field = "id",
                                            source_id = 0)
# Max profile id in the first dataset, used to separate the profiles in the next phases
separator_id = profiles1.map(lambda profile: profile.profile_id).max()
# Separators, used during blocking to understand from which dataset a profile belongs. It is an array because sparkER
# could work with multiple datasets
separator_ids = [separator_id]

In [126]:
type(separator_id)

int

In [127]:
separator_id

1362

In [128]:
profiles1.take(5)
profiles1.toDF().show(1364)

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{name, clickart ...| b000jz4hqo|         0|        0|
|[{name, ca intern...| b0006zf55o|         1|        0|
|[{name, noah's ar...| b00004tkvy|         2|        0|
|[{name, peachtree...| b000g80lqo|         3|        0|
|[{name, singing c...| b0006se5bq|         4|        0|
|[{name, emc retro...| b000ehpzv8|         5|        0|
|[{name, adobe aft...| b00021xhzw|         6|        0|
|[{name, acad upgr...| b000gzwjgc|         7|        0|
|[{name, mia's mat...| b0000dbykm|         8|        0|
|[{name, disney's ...| b00029bqa2|         9|        0|
|[{name, paper art...| b0007prnjo|        10|        0|
|[{name, nfs maest...| b000aazr5i|        11|        0|
|[{name, microsoft...| b000bhl1r8|        12|        0|
|[{name, spy sweep...| b000i82j80|        13|        0|
|[{name, domino de...| b00006hmwc|        14|   

In [129]:
a = profiles1.take(2)
a

[{'profile_id': 0, 'attributes': [{'key': 'name', 'value': 'clickart 950 000 - premier image pack (dvd-rom)'}, {'key': 'manufacturer', 'value': 'broderbund'}, {'key': 'price', 'value': '0'}], 'original_id': 'b000jz4hqo', 'source_id': 0},
 {'profile_id': 1, 'attributes': [{'key': 'name', 'value': 'ca international - arcserve lap/desktop oem 30pk'}, {'key': 'description', 'value': 'oem arcserve backup v11.1 win 30u for laptops and desktops'}, {'key': 'manufacturer', 'value': 'computer associates'}, {'key': 'price', 'value': '0'}], 'original_id': 'b0006zf55o', 'source_id': 0}]

Let's visualize a profile to check if they are correctly loaded

In [130]:
print(profiles1.take(1)[0])

{'profile_id': 0, 'attributes': [{'key': 'name', 'value': 'clickart 950 000 - premier image pack (dvd-rom)'}, {'key': 'manufacturer', 'value': 'broderbund'}, {'key': 'price', 'value': '0'}], 'original_id': 'b000jz4hqo', 'source_id': 0}


Loads the second dataset and extract the max id (it will be used later)

In [131]:
profiles2 = sparker.CSVWrapper.load_profiles('/Final_Project/amazon_google/GoogleProducts.csv', 
                                              start_id_from = separator_id+1, 
                                              real_id_field = "id",
                                            source_id = 1)
# Max profile id
max_profile_id = profiles2.map(lambda profile: profile.profile_id).max()

In [132]:
profiles2.count()

3226

In [133]:
profiles2.toDF().show(3226)

+--------------------+--------------------+----------+---------+
|          attributes|         original_id|profile_id|source_id|
+--------------------+--------------------+----------+---------+
|[{name, learning ...|http://www.google...|      1363|        1|
|[{name, superstar...|http://www.google...|      1364|        1|
|[{name, qb pos 6....|http://www.google...|      1365|        1|
|[{name, math miss...|http://www.google...|      1366|        1|
|[{name, productio...|http://www.google...|      1367|        1|
|[{name, video stu...|http://www.google...|      1368|        1|
|[{name, edius pro...|http://www.google...|      1369|        1|
|[{name, qb pos 6....|http://www.google...|      1370|        1|
|[{name, qb credit...|http://www.google...|      1371|        1|
|[{name, vegas mov...|http://www.google...|      1372|        1|
|[{name, qb pos 6....|http://www.google...|      1373|        1|
|[{name, ares risi...|http://www.google...|      1374|        1|
|[{name, evergirl:...|htt

Finally, concatenate the two RDDs

In [134]:
profiles = profiles1.union(profiles2)

In [135]:
profiles.take(2)

[{'profile_id': 0, 'attributes': [{'key': 'name', 'value': 'clickart 950 000 - premier image pack (dvd-rom)'}, {'key': 'manufacturer', 'value': 'broderbund'}, {'key': 'price', 'value': '0'}], 'original_id': 'b000jz4hqo', 'source_id': 0},
 {'profile_id': 1, 'attributes': [{'key': 'name', 'value': 'ca international - arcserve lap/desktop oem 30pk'}, {'key': 'description', 'value': 'oem arcserve backup v11.1 win 30u for laptops and desktops'}, {'key': 'manufacturer', 'value': 'computer associates'}, {'key': 'price', 'value': '0'}], 'original_id': 'b0006zf55o', 'source_id': 0}]

In [136]:
profiles.count()

4589

In [137]:
profiles.toDF().show(4590) #b000jz4hqo

+--------------------+--------------------+----------+---------+
|          attributes|         original_id|profile_id|source_id|
+--------------------+--------------------+----------+---------+
|[{name, clickart ...|          b000jz4hqo|         0|        0|
|[{name, ca intern...|          b0006zf55o|         1|        0|
|[{name, noah's ar...|          b00004tkvy|         2|        0|
|[{name, peachtree...|          b000g80lqo|         3|        0|
|[{name, singing c...|          b0006se5bq|         4|        0|
|[{name, emc retro...|          b000ehpzv8|         5|        0|
|[{name, adobe aft...|          b00021xhzw|         6|        0|
|[{name, acad upgr...|          b000gzwjgc|         7|        0|
|[{name, mia's mat...|          b0000dbykm|         8|        0|
|[{name, disney's ...|          b00029bqa2|         9|        0|
|[{name, paper art...|          b0007prnjo|        10|        0|
|[{name, nfs maest...|          b000aazr5i|        11|        0|
|[{name, microsoft...|   

### Groundtruth (optional)
If you have a groundtruth you can measure the performance of each step.

When you load the groundtruth it contains the original profiles IDs, it is necessary to convert it to use the IDs assigned to each profile by Spark.

In [138]:
# Loads the groundtruth, takes as input the path of the file and the names of the attributes that represent
# respectively the id of profiles of the first dataset and the id of profiles of the second dataset
gt = sparker.CSVWrapper.load_groundtruth('/Final_Project/amazon_google/Amzon_GoogleProducts_perfectMapping.csv',
                                        'idAmazon', 'idGoogleBase')

In [139]:
# gt.toDF().show(1400)

In [140]:
# # Converts the groundtruth by replacing original IDs with those given by Spark
new_gt = sparker.Converters.convert_groundtruth(gt, profiles1, profiles2)

In [141]:
new_gt

{(938, 2238),
 (598, 2318),
 (559, 1689),
 (10, 3316),
 (262, 2485),
 (924, 3916),
 (746, 3995),
 (1173, 2262),
 (758, 1754),
 (840, 4304),
 (980, 2212),
 (413, 4236),
 (771, 1729),
 (199, 3080),
 (186, 2881),
 (528, 3664),
 (268, 4491),
 (1124, 3546),
 (579, 3895),
 (646, 1670),
 (423, 3727),
 (469, 3843),
 (1286, 4102),
 (932, 1538),
 (1361, 2326),
 (1165, 2271),
 (1198, 1365),
 (66, 2937),
 (1046, 3711),
 (1356, 1987),
 (267, 3866),
 (902, 3968),
 (1164, 3479),
 (848, 4033),
 (882, 3909),
 (778, 3937),
 (1075, 2354),
 (91, 3007),
 (552, 1904),
 (534, 3433),
 (1103, 2803),
 (124, 4208),
 (1106, 2319),
 (799, 2167),
 (115, 3048),
 (1167, 2270),
 (667, 1575),
 (37, 3394),
 (193, 2887),
 (480, 3820),
 (1096, 3823),
 (589, 4401),
 (653, 1676),
 (1115, 2790),
 (1264, 3636),
 (711, 1766),
 (1251, 1896),
 (562, 2317),
 (111, 3055),
 (671, 1586),
 (1183, 2255),
 (1111, 1858),
 (1261, 3752),
 (126, 3183),
 (606, 3528),
 (764, 1750),
 (1136, 2296),
 (380, 4039),
 (321, 4483),
 (796, 3544),
 (1

In [142]:
# len(new_gt)

In [143]:
# # We can explore some pairs
# random.sample(new_gt, 5)

## Blocking
Now we can perform blocking.

By default sparkER performs token blocking, but it is possible to provide a different blocking function.

In the following example each token is splitted in ngrams of size 4 that are used for blocking.

In [144]:
# blocks = sparker.Blocking.create_blocks(profiles, separator_ids, 
#                                         blocking_method=sparker.BlockingKeysStrategies.ngrams_blocking,
#                                         ngram_size=4)
# print("Number of blocks",blocks.count())

In [145]:
# sparker.BlockingKeysStrategies.ngrams_blocking

In [146]:
# blocks.take(500)

### Let's continue by using the standard token blocking

In [147]:
blocks = sparker.Blocking.create_blocks(profiles, separator_ids)
print("Number of blocks",blocks.count())

Number of blocks 6834


In [148]:
blocks.take(1)

[{'block_id': 0, 'profiles': [{0, 512, 130, 641, 771, 900, 1285, 135, 392, 1288, 255, 511, 14, 142, 531, 787, 1175, 154, 797, 1183, 544, 293, 549, 423, 1191, 811, 685, 1326, 175, 431, 690, 437, 566, 949, 1333, 706, 580, 70, 582, 713, 460, 590, 591, 338, 211, 212, 600, 857, 218, 987, 732, 860, 223, 480, 865, 248, 871, 492, 748, 1008, 246, 1271, 632, 502, 251, 763, 767}, {3077, 3594, 2571, 3599, 3089, 2578, 1562, 2089, 3119, 1591, 3148, 1615, 1628, 2141, 2654, 2652, 2669, 3697, 2162, 2676, 1659, 2188, 1679, 1696, 2722, 3241, 4279, 1720, 1724, 3260, 1728, 1731, 1736, 2249, 1741, 2769, 2266, 3811, 4324, 4325, 4334, 2289, 3315, 4341, 3337, 4362, 1808, 3350, 1820, 1823, 2847, 2849, 3875, 3876, 4399, 2356, 2359, 3897, 2878, 1860, 2374, 4422, 3917, 4434, 1878, 3930, 2908, 2912, 2914, 1382, 1392, 3957, 4472, 1402, 2946, 1930, 2954, 1421, 3478, 1955, 1445, 1962, 4523, 1453, 3503, 1463, 4539, 3005, 2495, 3521, 3014, 1992, 3020, 2510, 2512, 3539, 2518, 4054, 2020, 1510, 2540, 3574, 3066, 3070}], '

## Block cleaning

sparkER implements two block cleaning strategies:

* Block purging: discard the largest blocks that involve too many comparisons, the parameter must be >= 1. A lower value mean a more aggressive purging.
* Block cleaning: removes for every profile the largest blocks in which it appears. The parameter is in range ]0, 1\[. A lower value mean a more aggressive cleaning.

In [149]:
# Perfoms the purging
blocks_purged = sparker.BlockPurging.block_purging(blocks, 1.025)

In [150]:
blocks_purged.take(500)

[{'block_id': 0, 'profiles': [{0, 512, 130, 641, 771, 900, 1285, 135, 392, 1288, 255, 511, 14, 142, 531, 787, 1175, 154, 797, 1183, 544, 293, 549, 423, 1191, 811, 685, 1326, 175, 431, 690, 437, 566, 949, 1333, 706, 580, 70, 582, 713, 460, 590, 591, 338, 211, 212, 600, 857, 218, 987, 732, 860, 223, 480, 865, 248, 871, 492, 748, 1008, 246, 1271, 632, 502, 251, 763, 767}, {3077, 3594, 2571, 3599, 3089, 2578, 1562, 2089, 3119, 1591, 3148, 1615, 1628, 2141, 2654, 2652, 2669, 3697, 2162, 2676, 1659, 2188, 1679, 1696, 2722, 3241, 4279, 1720, 1724, 3260, 1728, 1731, 1736, 2249, 1741, 2769, 2266, 3811, 4324, 4325, 4334, 2289, 3315, 4341, 3337, 4362, 1808, 3350, 1820, 1823, 2847, 2849, 3875, 3876, 4399, 2356, 2359, 3897, 2878, 1860, 2374, 4422, 3917, 4434, 1878, 3930, 2908, 2912, 2914, 1382, 1392, 3957, 4472, 1402, 2946, 1930, 2954, 1421, 3478, 1955, 1445, 1962, 4523, 1453, 3503, 1463, 4539, 3005, 2495, 3521, 3014, 1992, 3020, 2510, 2512, 3539, 2518, 4054, 2020, 1510, 2540, 3574, 3066, 3070}], '

In [151]:
# Performs the cleaning
(profile_blocks, profile_blocks_filtered, blocks_after_filtering) = sparker.BlockFiltering.block_filtering_quick(blocks_purged, 0.8, separator_ids)

In [152]:
blocks_purged.take(500)

[{'block_id': 0, 'profiles': [{0, 512, 130, 641, 771, 900, 1285, 135, 392, 1288, 255, 511, 14, 142, 531, 787, 1175, 154, 797, 1183, 544, 293, 549, 423, 1191, 811, 685, 1326, 175, 431, 690, 437, 566, 949, 1333, 706, 580, 70, 582, 713, 460, 590, 591, 338, 211, 212, 600, 857, 218, 987, 732, 860, 223, 480, 865, 248, 871, 492, 748, 1008, 246, 1271, 632, 502, 251, 763, 767}, {3077, 3594, 2571, 3599, 3089, 2578, 1562, 2089, 3119, 1591, 3148, 1615, 1628, 2141, 2654, 2652, 2669, 3697, 2162, 2676, 1659, 2188, 1679, 1696, 2722, 3241, 4279, 1720, 1724, 3260, 1728, 1731, 1736, 2249, 1741, 2769, 2266, 3811, 4324, 4325, 4334, 2289, 3315, 4341, 3337, 4362, 1808, 3350, 1820, 1823, 2847, 2849, 3875, 3876, 4399, 2356, 2359, 3897, 2878, 1860, 2374, 4422, 3917, 4434, 1878, 3930, 2908, 2912, 2914, 1382, 1392, 3957, 4472, 1402, 2946, 1930, 2954, 1421, 3478, 1955, 1445, 1962, 4523, 1453, 3503, 1463, 4539, 3005, 2495, 3521, 3014, 1992, 3020, 2510, 2512, 3539, 2518, 4054, 2020, 1510, 2540, 3574, 3066, 3070}], '

In [153]:
print("Number of blocks_purged",blocks_purged.count())

Number of blocks_purged 6818


If you have the groundtruth, after every blocking step it is possible to check which are the performance of the blocking collection.

In [154]:
recall, precision, cmp_n = sparker.Utils.get_statistics(blocks_after_filtering, max_profile_id, new_gt, separator_ids)

print("Recall", recall)
print("Precision", precision)
print("Number of comparisons", cmp_n)

Recall 0.9984615384615385
Precision 0.0009325663968594497
Number of comparisons 1391858


## Meta-blocking
Meta-blocking can be used to further refine the block collection removing superfluous comparisons.

SparkER implements different kind of meta-blocking algorithms, you can find the descriptions in our paper.


For every partition of the RDD the pruning algorithm returns as output a triplet that contains:

* The number of edges
* The number of matches (only if the groundtruth is provided)
* The retained edges

To perform the meta-blocking first some data structures have to be created.

In [155]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [156]:
block_index_map = blocks_after_filtering.map(lambda b : (b.block_id, b.profiles)).collectAsMap()
block_index = sc.broadcast(block_index_map)

# This is only needed for certain weight measures
profile_blocks_size_index = sc.broadcast(profile_blocks_filtered.map(lambda pb : (pb.profile_id, len(pb.blocks))).collectAsMap())

# # Broadcasted groundtruth
gt_broadcast = sc.broadcast(new_gt)

### Weighted Node Pruning

In [157]:
# results = sparker.WNP.wnp(
#                           profile_blocks_filtered,
#                           block_index,
#                           max_profile_id,
#                           separator_ids,
#                           weight_type=sparker.WeightTypes.CBS,
#                           #groundtruth=gt_broadcast,
#                           profile_blocks_size_index=profile_blocks_size_index,
#                           comparison_type=sparker.ComparisonTypes.OR
#                          )
# num_edges = results.map(lambda x: x[0]).sum()
# num_matches = results.map(lambda x: x[1]).sum()
# # print("Recall", num_matches/len(new_gt))
# # print("Precision", num_matches/num_edges)
# print("Number of comparisons",num_edges)

The number of edges
The number of matches (only if the groundtruth is provided)
The retained edges

In [158]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show()

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|          7|            1|[{0, 4297, 2}, {0...|
|       1393|            1|[{512, 4068, 3}, ...|
|         16|            1|[{130, 3077, 2}, ...|
|        488|            0|[{900, 2481, 2}, ...|
|        128|            0|[{392, 3469, 2}, ...|
|        144|            1|[{1288, 4098, 2},...|
|         37|            0|[{14, 4537, 2}, {...|
|         40|            0|[{142, 3172, 9}, ...|
|        179|            0|[{154, 3588, 4}, ...|
|        547|            1|[{544, 2923, 2}, ...|
|        750|            1|[{1326, 4084, 4},...|
|         84|            1|[{690, 1564, 2}, ...|
|        100|            1|[{566, 2153, 4}, ...|
|        667|            1|[{706, 2006, 4}, ...|
|       1223|            1|[{580, 1538, 5}, ...|
|          4|            0|[{70, 3347, 2}, {...|
|        157|            1|[{582, 4101, 2}, ...|
|         76|       

### Reciprocal Weighted Node Pruning

In [159]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.AND
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Recall 0.9776923076923076
Precision 0.0031519377845671604
Number of comparisons 403244


In [160]:
# newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
# results.toDF(newColumns).show(10)

### Weighted Edge Pruning

In [161]:
# results = sparker.WEP.wep(
#                           profile_blocks_filtered,
#                           block_index,
#                           max_profile_id,
#                           separator_ids,
#                           weight_type=sparker.WeightTypes.CBS,
#                           groundtruth=gt_broadcast,
#                           profile_blocks_size_index=profile_blocks_size_index
#                          )
# num_edges = results.map(lambda x: x[0]).sum()
# num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
# print("Number of comparisons",num_edges)

In [162]:
# newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
# results.toDF(newColumns).show(10)

### Cardinality Node Pruning

In [163]:
# results = sparker.CNP.cnp(
#                           blocks_after_filtering,
#                           profiles.count(),
#                           profile_blocks_filtered,
#                           block_index,
#                           max_profile_id,
#                           separator_ids,
#                           weight_type=sparker.WeightTypes.CBS,
#                           groundtruth=gt_broadcast,
#                           profile_blocks_size_index=profile_blocks_size_index,
#                           comparison_type=sparker.ComparisonTypes.OR
#                          )
# num_edges = results.map(lambda x: x[0]).sum()
# num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
# print("Number of comparisons",num_edges)

In [164]:
# newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
# results.toDF(newColumns).show(10)

In [165]:
# newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
# results.toDF(newColumns).show(10)

### Reciprocal Cardinality Node Pruning

In [166]:
# results = sparker.CNP.cnp(
#                           blocks_after_filtering,
#                           profiles.count(),
#                           profile_blocks_filtered,
#                           block_index,
#                           max_profile_id,
#                           separator_ids,
#                           weight_type=sparker.WeightTypes.CBS,
#                           groundtruth=gt_broadcast,
#                           profile_blocks_size_index=profile_blocks_size_index,
#                           comparison_type=sparker.ComparisonTypes.AND
#                          )
# num_edges = results.map(lambda x: x[0]).sum()
# num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
# print("Number of comparisons",num_edges)

In [167]:
# newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
# results.toDF(newColumns).show(10)

### Cardinality Edge Pruning

In [168]:
# results = sparker.CEP.cep(
#                           profile_blocks_filtered,
#                           block_index,
#                           max_profile_id,
#                           separator_ids,
#                           weight_type=sparker.WeightTypes.CBS,
#                           #groundtruth=gt_broadcast,
#                           profile_blocks_size_index=profile_blocks_size_index
#                          )
# num_edges = results.map(lambda x: x[0]).sum()
# num_matches = results.map(lambda x: x[1]).sum()
# # print("Recall", num_matches/len(new_gt))
# # print("Precision", num_matches/num_edges)
# print("Number of comparisons",num_edges)

In [169]:
# newColumns = ["no_of_edges","true_matches_gt","retaind_edges"]
# result = results.toDF(newColumns)
# result.show()

In [170]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show(truncate=False)

+-----------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Collecting edges after meta-blocking
As mentioned before, the third element of the tuples returned by the meta-blocking contains the edges.


Edges are weighted according to the weight strategy provided to the meta-blocking.

In [171]:
edges = results.flatMap(lambda x: x[2])

edges.count()

403244

In [172]:
newColumns = ["profile_id","neighbor_id","neighbor_weight"]
ans = edges.toDF(newColumns)
# ans.sort_values(by=['neighbor_weight'])    #show(50000)

In [173]:

ans = spark.createDataFrame(data = edges, schema = newColumns)

In [174]:
ans.show(50000)

+----------+-----------+---------------+
|profile_id|neighbor_id|neighbor_weight|
+----------+-----------+---------------+
|         0|       4297|              2|
|         0|       3387|              2|
|         0|       3241|              5|
|         0|       1396|              2|
|         0|       1955|              2|
|         0|       2249|              2|
|         0|       2393|              2|
|       512|       3681|              3|
|       512|       4228|              6|
|       512|       3626|              8|
|       512|       3852|              9|
|       512|       3628|              9|
|       512|       2190|              4|
|       512|       4525|              4|
|       512|       3566|              3|
|       512|       2897|              6|
|       512|       3315|             12|
|       512|       3892|              3|
|       512|       3477|              3|
|       512|       4278|              6|
|       512|       3895|              3|
|       512|    

In [175]:
max_by_group = (edges
  .map(lambda x: (x[0], x))  
   .reduceByKey(lambda x1, x2: max(x1, x2, key=lambda x: x[-1]))
   .values())

In [176]:
max_by_group.take(30)

[(0, 3241, 5),
 (512, 3315, 12),
 (130, 2517, 3),
 (900, 4399, 9),
 (392, 4325, 26),
 (1288, 1856, 15),
 (14, 3337, 5),
 (142, 3172, 9),
 (154, 3259, 8),
 (544, 2518, 6),
 (1326, 2041, 23),
 (690, 1615, 6),
 (566, 1720, 7),
 (706, 2792, 9),
 (580, 4455, 16),
 (70, 2524, 2),
 (582, 1820, 5),
 (460, 3875, 11),
 (590, 1820, 7),
 (338, 4422, 22),
 (212, 3135, 15),
 (600, 2139, 15),
 (218, 1474, 9),
 (732, 1795, 16),
 (860, 2867, 16),
 (480, 3820, 24),
 (492, 1392, 5),
 (632, 3315, 12),
 (748, 3503, 6),
 (1008, 3852, 19)]

In [177]:
dataCollect = max_by_group.collect()
df =  pd.DataFrame(dataCollect)
df

0     1   2
0        0  3241   5
1      512  3315  12
2      130  2517   3
3      900  4399   9
4      392  4325  26
...    ...   ...  ..
1356   665  1572   4
1357   131  4470   3
1358  1003  3651   3
1359   977  4033   3
1360    99  1536   3

[1361 rows x 3 columns]

In [178]:
df.rename(columns = {0:'P_id', 1:'N_id',
                              2:'weight'}, inplace = True)
df.head()

P_id  N_id  weight
0     0  3241       5
1   512  3315      12
2   130  2517       3
3   900  4399       9
4   392  4325      26

In [179]:
n = df.filter(["P_id","N_id","weight"])
n = pd.DataFrame(n)
n

P_id  N_id  weight
0        0  3241       5
1      512  3315      12
2      130  2517       3
3      900  4399       9
4      392  4325      26
...    ...   ...     ...
1356   665  1572       4
1357   131  4470       3
1358  1003  3651       3
1359   977  4033       3
1360    99  1536       3

[1361 rows x 3 columns]

In [180]:
avg_val = n["weight"].mean()
avg_val = round(avg_val,0)
avg_val

10.0

In [181]:
n["N_id"] = n["N_id"]-(separator_id+1)
new_df = n[(n['weight']>avg_val)]
new_df

P_id  N_id  weight
1      512  1952      12
4      392  2962      26
5     1288   493      15
10    1326   678      23
14     580  3092      16
...    ...   ...     ...
1313   401  2948      11
1316    97  1637      16
1326   299  2964      16
1330   447  2157      14
1331   983   821      23

[540 rows x 3 columns]

In [182]:
new_df["c_id"] = [x for x in range(0,len(new_df))]
new_df

/tmp/ipykernel_4771/1762727809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["c_id"] = [x for x in range(0,len(new_df))]


P_id  N_id  weight  c_id
1      512  1952      12     0
4      392  2962      26     1
5     1288   493      15     2
10    1326   678      23     3
14     580  3092      16     4
...    ...   ...     ...   ...
1313   401  2948      11   535
1316    97  1637      16   536
1326   299  2964      16   537
1330   447  2157      14   538
1331   983   821      23   539

[540 rows x 4 columns]

In [183]:
# new_df = a[(a['e']>7)]
# new_df

In [184]:
col = ["P_id","N_id","weight","cluster_id"]
df_weight = spark.createDataFrame(new_df, schema=col)
df_weight.show(3000)

+----+----+------+----------+
|P_id|N_id|weight|cluster_id|
+----+----+------+----------+
| 512|1952|    12|         0|
| 392|2962|    26|         1|
|1288| 493|    15|         2|
|1326| 678|    23|         3|
| 580|3092|    16|         4|
| 460|2512|    11|         5|
| 338|3059|    22|         6|
| 212|1772|    15|         7|
| 600| 776|    15|         8|
| 732| 432|    16|         9|
| 860|1504|    16|        10|
| 480|2457|    24|        11|
| 632|1952|    12|        12|
|1008|2489|    19|        13|
|  10|1953|    25|        14|
|  44|2062|    14|        15|
|  62|2127|    12|        16|
|  76|1897|    16|        17|
| 100|1542|    19|        18|
| 106|1979|    21|        19|
| 114|1070|    14|        20|
| 138|1984|    19|        21|
| 156|2479|    21|        22|
| 160|1913|    15|        23|
| 168|2160|    12|        24|
| 198|  64|    18|        25|
| 210| 126|    13|        26|
| 240| 723|    17|        27|
| 270|3003|    22|        28|
| 272|2603|    11|        29|
| 288|3006

In [185]:
ss = SparkSession(sc)


In [193]:
df1 = ss.read.option("header",True)\
        .option("inferSchema",True)\
        .csv('file:///home/aditi/BigData/AAI_Project/CODE/Amazon.csv')
df1.show(5)

+----------+--------------------+--------------------+--------------------+------+
|        id|                name|         description|        manufacturer| price|
+----------+--------------------+--------------------+--------------------+------+
|b000jz4hqo|clickart 950 000 ...|                null|          broderbund|   0.0|
|b0006zf55o|ca international ...|oem arcserve back...| computer associates|   0.0|
|b00004tkvy|noah's ark activi...|                null|  victory multimedia|   0.0|
|b000g80lqo|peachtree by sage...|peachtree premium...|       sage software|599.99|
|b0006se5bq|singing coach unl...|singing coach unl...|carry-a-tune tech...| 99.99|
+----------+--------------------+--------------------+--------------------+------+
only showing top 5 rows



In [194]:
rdd_df = df1.rdd.zipWithIndex()

In [195]:
df1 = rdd_df.toDF()

In [196]:
df1.show()

+--------------------+---+
|                  _1| _2|
+--------------------+---+
|{b000jz4hqo, clic...|  0|
|{b0006zf55o, ca i...|  1|
|{b00004tkvy, noah...|  2|
|{b000g80lqo, peac...|  3|
|{b0006se5bq, sing...|  4|
|{b000ehpzv8, emc ...|  5|
|{b00021xhzw, adob...|  6|
|{b000gzwjgc, acad...|  7|
|{b0000dbykm, mia'...|  8|
|{b00029bqa2, disn...|  9|
|{b0007prnjo, pape...| 10|
|{b000aazr5i, nfs ...| 11|
|{b000bhl1r8, micr...| 12|
|{b000i82j80, spy ...| 13|
|{b00006hmwc, domi...| 14|
|{b000in6u62, omni...| 15|
|{b000083k56, comp...| 16|
|{b00006hvvo, upg ...| 17|
|{b0000ycfcw, the ...| 18|
|{b00066dd5m, scho...| 19|
+--------------------+---+
only showing top 20 rows



In [197]:
df1 = df1.withColumn('id', df1['_1'].getItem("id"))
df1 = df1.withColumn('name', df1['_1'].getItem("name"))
df1 = df1.withColumn('description', df1['_1'].getItem("description"))
df1 = df1.withColumn('manufacturer', df1['_1'].getItem("manufacturer"))
df1 = df1.withColumn('price', df1['_1'].getItem("price"))
df1 = df1.withColumnRenamed("_2" , "new_id_1")


In [198]:
df1 = df1.drop("_1")
df1.show()

+--------+----------+--------------------+--------------------+--------------------+------+
|new_id_1|        id|                name|         description|        manufacturer| price|
+--------+----------+--------------------+--------------------+--------------------+------+
|       0|b000jz4hqo|clickart 950 000 ...|                null|          broderbund|   0.0|
|       1|b0006zf55o|ca international ...|oem arcserve back...| computer associates|   0.0|
|       2|b00004tkvy|noah's ark activi...|                null|  victory multimedia|   0.0|
|       3|b000g80lqo|peachtree by sage...|peachtree premium...|       sage software|599.99|
|       4|b0006se5bq|singing coach unl...|singing coach unl...|carry-a-tune tech...| 99.99|
|       5|b000ehpzv8|emc retrospect 7....|emc retrospect 7....|               dantz|   0.0|
|       6|b00021xhzw|adobe after effec...|upgrade only; ins...|               adobe|499.99|
|       7|b000gzwjgc|acad upgrade drag...|- marketing infor...|     nuance acade

In [199]:
df2 = ss.read.option("header",True)\
        .option("inferSchema",True)\
        .csv('/Final_Project/amazon_google/GoogleProducts.csv')
df2.show(5)

+--------------------+--------------------+--------------------+--------------+------+
|                  id|                name|         description|  manufacturer| price|
+--------------------+--------------------+--------------------+--------------+------+
|http://www.google...|learning quickboo...|learning quickboo...|        intuit| 38.99|
|http://www.google...|superstart! fun w...|fun with reading ...|          null|  8.49|
|http://www.google...|qb pos 6.0 basic ...|qb pos 6.0 basic ...|        intuit|637.99|
|http://www.google...|math missions: th...|save spectacle ci...|          null| 12.95|
|http://www.google...|production prem c...|adobe cs3 product...|adobe software|805.99|
+--------------------+--------------------+--------------------+--------------+------+
only showing top 5 rows



In [200]:
rdd_df2= df2.rdd.zipWithIndex()
df2 = rdd_df2.toDF()
df2.show()

+--------------------+---+
|                  _1| _2|
+--------------------+---+
|{http://www.googl...|  0|
|{http://www.googl...|  1|
|{http://www.googl...|  2|
|{http://www.googl...|  3|
|{http://www.googl...|  4|
|{http://www.googl...|  5|
|{http://www.googl...|  6|
|{http://www.googl...|  7|
|{http://www.googl...|  8|
|{http://www.googl...|  9|
|{http://www.googl...| 10|
|{http://www.googl...| 11|
|{http://www.googl...| 12|
|{http://www.googl...| 13|
|{http://www.googl...| 14|
|{http://www.googl...| 15|
|{http://www.googl...| 16|
|{http://www.googl...| 17|
|{http://www.googl...| 18|
|{http://www.googl...| 19|
+--------------------+---+
only showing top 20 rows



In [201]:
df2 = df2.withColumn('id', df2['_1'].getItem("id"))
df2 = df2.withColumn('name', df2['_1'].getItem("name"))
df2 = df2.withColumn('description', df2['_1'].getItem("description"))
df2 = df2.withColumn('manufacturer', df2['_1'].getItem("manufacturer"))
df2 = df2.withColumn('price', df2['_1'].getItem("price"))
df2 = df2.withColumnRenamed("_2" , "new_id_1")

In [202]:
df2 = df2.drop("_1")
df2.show()

+--------+--------------------+--------------------+--------------------+--------------------+-------+
|new_id_1|                  id|                name|         description|        manufacturer|  price|
+--------+--------------------+--------------------+--------------------+--------------------+-------+
|       0|http://www.google...|learning quickboo...|learning quickboo...|              intuit|  38.99|
|       1|http://www.google...|superstart! fun w...|fun with reading ...|                null|   8.49|
|       2|http://www.google...|qb pos 6.0 basic ...|qb pos 6.0 basic ...|              intuit| 637.99|
|       3|http://www.google...|math missions: th...|save spectacle ci...|                null|  12.95|
|       4|http://www.google...|production prem c...|adobe cs3 product...|      adobe software| 805.99|
|       5|http://www.google...|video studio 11 plus|corel video studi...|   corel corporation| 103.99|
|       6|http://www.google...|         edius pro 4|whether you are w...|

In [203]:
common_df = df1.join(df_weight,df1.new_id_1 == df_weight.P_id,"inner")

In [204]:
pd_common_df = common_df.toPandas()
pd_common_df

new_id_1          id                                               name  \
0         270  b00069e7ko                     adobe acrobat 7.0 professional   
1         198  b000fqvxge  scansoft paperport professional 11 - 5 user li...   
2         600  b0008glg1i          apple .mac 4.0 online service family pack   
3         212  b000ju9nay             diskeeper 2007 pro premier 10-lic pack   
4         138  b0000swyw4     sonicwall gms standard edition 10 node license   
..        ...         ...                                                ...   
535       793  b000lu89uc                        hoyle south beach solitaire   
536       969  b0000899lt        talk now! learn afrikaans - beginning level   
537       717  b00076ywii  learn to play chess with fritz & chesster 2 (mac)   
538      1243  b000lff13k                                               prey   
539      1033  b000hpf7p6                         pc professor fuji's kakuro   

                                           description  \
0    acrobat 7.0 professional is the advanced way t...   
1    - marketing information: paperport professiona...   
2    simply put .mac is your life on the internet. ...   
3    - marketing information: diskeeper pro premier...   
4    today enterprises and service providers face i...   
..                                                 ...   
535  latin tempos and art deco style energize this ...   
536  talk now! beginners afrikaans is the perfect w...   
537  learn to play chess with fritz & chesster lets...   
538  prey is a groundbreaking first-person shooter ...   
539  professor fuji's kakuro offers the most puzzle...   

                   manufacturer    price  P_id  N_id  weight  cluster_id  
0                         adobe   449.99   270  3003      22          28  
1    nuance communications inc.   949.99   198    64      18          25  
2                         apple   179.95   600   776      15           8  
3         diskeeper corporation  1372.21   212  1772      15           7  
4                     sonicwall     0.00   138  1984      19          21  
..                          ...      ...   ...   ...     ...         ...  
535                      encore     9.99   793  2570      18         532  
536                    eurotalk    29.99   969   837      21         505  
537                  viva media    29.99   717   500      13         522  
538                 aspyr media    39.99  1243   616      20         515  
539             encore software     9.99  1033   762      15         527  

[540 rows x 10 columns]

In [205]:
common_df_2 = common_df.join(df2,common_df.N_id == df2.new_id_1,"inner")
pd_common_df_2 = common_df_2.toPandas()
pd_common_df_2.iloc[30:40]

new_id_1          id                                               name  \
30       237  b0002tpg1w        persoft upg smarterm office v12.1 ( 73098 )   
31      1064  b000gkpds8  adventure workshop 4th-6th grade 7th ed sb cs ...   
32       760  b000v05f4k  panda software antivirus & firewall 2008 - win...   
33       316  b000ndemb2            f-secure internet security 2007 - 3 pcs   
34       328  b000h229hi                          state standards - grade 8   
35       816  b000barbaq  microsoft visual studio team suite 2005 with m...   
36       870  b00006y0es                           tiny term emulator v4.3x   
37       490  b000bt8trg  microsoft visual studio professional 2005 upgrade   
38       735  b000egoe9s          adventure workshop preschool (jewel case)   
39       761  b000bt559q  microsoft visual studio team edition for softw...   

                                          description  \
30  smarterm pc-to-host connectivity software has ...   
31  introduce your child to an exciting on-screen ...   
32  now you can be sure that all the information o...   
33  f-secure internet security 2007 provides a com...   
34  state standards grade 8 has everything you nee...   
35  microsoft visual studio team suite 2005 progra...   
36  - marketing information: tinyterm provides acc...   
37  microsoft visual studio professional 2005 is o...   
38  adventure workshop 7 preschool - 1st grade fea...   
39  in the past testing tools were kept separate f...   

                 manufacturer     price  P_id  N_id  weight  cluster_id  \
30                    persoft      0.00   237   724      22         304   
31            encore software     19.99  1064  2355      18         234   
32             panda software      0.00   760  2489      11          57   
33  global marketing partners     79.99   316  2489      15         161   
34       topics entertainment     29.95   328  2966      18         191   
35                  microsoft  10949.00   816  2681      21          60   
36           century software    219.63   870  2488      24         164   
37         microsoft software    549.00   490  2681      12          42   
38            encore software     29.99   735  2355      13         514   
39         microsoft software   2309.00   761  2681      12         479   

    new_id_1                                                 id  \
30       724  http://www.google.com/base/feeds/snippets/1306...   
31      2355  http://www.google.com/base/feeds/snippets/5940...   
32      2489  http://www.google.com/base/feeds/snippets/1837...   
33      2489  http://www.google.com/base/feeds/snippets/1837...   
34      2966  http://www.google.com/base/feeds/snippets/1051...   
35      2681  http://www.google.com/base/feeds/snippets/3137...   
36      2488  http://www.google.com/base/feeds/snippets/1244...   
37      2681  http://www.google.com/base/feeds/snippets/3137...   
38      2355  http://www.google.com/base/feeds/snippets/5940...   
39      2681  http://www.google.com/base/feeds/snippets/3137...   

                                                 name  \
30  73098 smarterm office - ( v. 12.1 ) - upgrade ...   
31  encore software 12411 - encore adventure works...   
32  symantec 10725608 - norton internet security 2...   
33  symantec 10725608 - norton internet security 2...   
34                          state standards - grade 6   
35  microsoft 122-00411 - visual studio 2005 team ...   
36  century software tt-1 - century tinyterm v.4.3...   
37  microsoft 122-00411 - visual studio 2005 team ...   
38  encore software 12411 - encore adventure works...   
39  microsoft 122-00411 - visual studio 2005 team ...   

                                          description manufacturer    price  
30  esker software 73098 : usually ships in 24 hou...         None    85.89  
31  encore software 12411 : introduce your child t...         None    17.97  
32  symantec 10725608 : stay protected from the la...         None    64.97  
33

In [206]:
pd_common_df_2.iloc[20:30]

new_id_1          id                                           name  \
20       447  b0000ycfdg                             simple movie maker   
21        79  b000i84dsy       ca internet security suite 2007 - 3 user   
22       377  b000nknb62  pc cillin internet sec 2007 lics large box 3u   
23      1315  b00065hjvc                        solitaire antics deluxe   
24       232  b000icpcxa                 printshop home & office labels   
25       911  b0008g2bvm                                        obscure   
26      1129  b0002jtvng                  bias deck le 3.5 macintosh cd   
27      1193  b000lwfbem                                mask pro 4.0 1u   
28       417  b000gs9tq2                                  pressure pop!   
29       332  b000bcgfe2                               spy sweeper [lb]   

                                          description  \
20  lights!&nbsp; camera!&nbsp; interactive!produc...   
21  internet security 2007 gives you comprehensive...   
22  pc-cillin internet security 2007 combines awar...   
23  50 different games of solitaire includes tours...   
24  design label and organize like a pro! product ...   
25  obscure takes the survival-horror gaming you l...   
26  if you want to record music and audio like a p...   
27  in any given image you may need more than one ...   
28  click-popping frenzy!product informationthe pr...   
29  spy sweeper 4.5 defends your pc and your perso...   

                manufacturer   price  P_id  N_id  weight  cluster_id  \
20      topics entertainment   19.99   447  2157      14         538   
21       computer associates   69.95    79  3144      11         432   
22               trend micro   49.95   377  3144      12         481   
23         masque publishing    0.00  1315   656      26         360   
24                broderbund   19.95   232  2878      23         105   
25  dreamcatcher interactive   19.99   911   167      13         495   
26                      bias   99.00  1129  2175      16         470   
27            onone software  159.95  1193  2672      12         354   
28     selectsoft publishing   19.95   417  2882      16         497   
29          webroot software   29.99   332  1547      11         266   

    new_id_1                                                 id  \
20      2157  http://www.google.com/base/feeds/snippets/1372...   
21      3144  http://www.google.com/base/feeds/snippets/1419...   
22      3144  http://www.google.com/base/feeds/snippets/1419...   
23       656  http://www.google.com/base/feeds/snippets/1753...   
24      2878  http://www.google.com/base/feeds/snippets/1161...   
25       167  http://www.google.com/base/feeds/snippets/1827...   
26      2175  http://www.google.com/base/feeds/snippets/5572...   
27      2672  http://www.google.com/base/feeds/snippets/5100...   
28      2882  http://www.google.com/base/feeds/snippets/1019...   
29      1547  http://www.google.com/base/feeds/snippets/1838...   

                                                 name  \
20                                 simple movie maker   
21               zonealarm(r) internet security suite   
22               zonealarm(r) internet security suite   
23          masque publishing solitaire antics deluxe   
24                     printshop home & office labels   
25                   dreamcatcher interactive obscure   
26  bias deck 3.5 le digital audio recording softw...   
27  onone software mpr-40212 mask pro 4.0 upgrade ...   
28                                      pressure pop!   
29                            spy sweeper (small box)   

                                          description    manufacturer  price  
20  leap from camcorder photography to cinematic p...            None   12.9  
21  looking for easy-to-use protection against hac...            None  49.99  
22  looking for easy-to-use protection against hac...            None  49.99  
23  50 different games of solitaire includes tours...            None   8.4

In [207]:
pd_common_df_2.to_csv('/home/aditi/BigData/AAI_Project/dataset/Amazon_Output.csv', header=True, index=False)

########################################################################
########################################################################

In [208]:
common_df = df1.join(df_weight,df1.new_id_1 == df_weight.P_id,"inner")

In [209]:
common_df.printSchema()

root
 |-- new_id_1: long (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- price: double (nullable = true)
 |-- P_id: long (nullable = true)
 |-- N_id: long (nullable = true)
 |-- weight: long (nullable = true)
 |-- cluster_id: long (nullable = true)



In [210]:
common_N = df2.join(df_weight,df2.new_id_1 == df_weight.N_id,"inner")

In [211]:
common_N.printSchema()

root
 |-- new_id_1: long (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- price: string (nullable = true)
 |-- P_id: long (nullable = true)
 |-- N_id: long (nullable = true)
 |-- weight: long (nullable = true)
 |-- cluster_id: long (nullable = true)



In [212]:
import functools

def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)
  
  
# unionAll
result3 = unionAll([common_df, common_N])
result3.show()

+--------+----------+--------------------+--------------------+--------------------+------+----+----+------+----------+
|new_id_1|        id|                name|         description|        manufacturer| price|P_id|N_id|weight|cluster_id|
+--------+----------+--------------------+--------------------+--------------------+------+----+----+------+----------+
|      26|b000o24l3q|adobe premiere pr...|note: this is the...|               adobe| 299.0|  26| 190|    14|       113|
|     964|b00005birs|              math 2|based on the scho...|         school zone|  9.99| 964| 871|    11|       227|
|     418|b000ema3zq|essential busines...|everything you ne...|            valusoft| 19.99| 418|2884|    19|        37|
|     541|b000hcz8ey|microsoft onenote...|office onenote 20...|           microsoft| 99.95| 541|3199|    12|       469|
|    1010|b000hjqyc2|roxio easy media ...|burn music to cd ...|               roxio| 99.95|1010|1160|    13|       140|
|    1258|b00006sijr|punch! home desig..

In [213]:
final = result3.toPandas()

In [214]:
final = final.drop(["P_id","N_id","weight"], axis=1)
final.head()

new_id_1          id                                               name  \
0       270  b00069e7ko                     adobe acrobat 7.0 professional   
1       198  b000fqvxge  scansoft paperport professional 11 - 5 user li...   
2       600  b0008glg1i          apple .mac 4.0 online service family pack   
3       212  b000ju9nay             diskeeper 2007 pro premier 10-lic pack   
4       138  b0000swyw4     sonicwall gms standard edition 10 node license   

                                         description  \
0  acrobat 7.0 professional is the advanced way t...   
1  - marketing information: paperport professiona...   
2  simply put .mac is your life on the internet. ...   
3  - marketing information: diskeeper pro premier...   
4  today enterprises and service providers face i...   

                 manufacturer    price  cluster_id  
0                       adobe   449.99          28  
1  nuance communications inc.   949.99          25  
2                       apple   179.95           8  
3       diskeeper corporation  1372.21           7  
4                   sonicwall      0.0          21

In [215]:
final = final.groupby(['cluster_id','id'])

In [220]:
final.first()

new_id_1  \
cluster_id id                                                             
0          b00022ptt8                                               512   
           http://www.google.com/base/feeds/snippets/18434...      1952   
1          b0002j6i2s                                               392   
           http://www.google.com/base/feeds/snippets/11142...      2962   
2          b000o39qyo                                              1288   
...                                                                 ...   
537        http://www.google.com/base/feeds/snippets/11194...      2964   
538        b0000ycfdg                                               447   
           http://www.google.com/base/feeds/snippets/13728...      2157   
539        b0009jhv5e                                               983   
           http://www.google.com/base/feeds/snippets/16721...       821   

                                                                                                            name  \
cluster_id id                                                                                                      
0          b00022ptt8                                          microsoft windows xp professional upgrade with...   
           http://www.google.com/base/feeds/snippets/18434...  symantec 10517904 - client security v.3.1 busi...   
1          b0002j6i2s                                                            clifford reading pack (dvd box)   
           http://www.google.com/base/feeds/snippets/11142...            clifford the big red dog - reading pack   
2          b000o39qyo                                                       city of heroes good vs. evil edition   
...                                                                                                          ...   
537        http://www.google.com/base/feeds/snippets/11194...                                         wacky farm   
538        b0000ycfdg                                                                         simple movie maker   
           http://www.google.com/base/feeds/snippets/13728...                                 simple movie maker   
539        b0009jhv5e                                                                             cook'n low fat   
           http://www.google.com/base/feeds/snippets/16721...                 dvo enterprises inc. cookn low fat   

                                                                                                     description  \
cluster_id id                                                                                                      
0          b00022ptt8                                          with windows xp professional edition with serv...   
           http://www.google.com/base/feeds/snippets/18434...  symantec 10517904 : symantec client security b...   
1          b0002j6i2s                                          pbs' famous scarlet hound gives pause to teach...   
           http://www.google.com/base/feeds/snippets/11142...  pbs' famous scarlet hound gives pause to teach...   
2          b000o39qyo                                          when city of heroes first launched the world o...   
...                                                                                                          ...   
537        http://www.google.com/base/feeds/snippets/11194...  moo-ve over macdonald - wacky farm is here! in...   
538        b0000ycfdg                                          lights!&nbsp; camera!&nbsp; interactive!produc...   
           http://www.google.com/base/feeds/snippets/13728...  leap from camcorder photography to cinematic p...   
539        b0009jhv5e                                          healthy eating can be easy and convenient and ...   
           http://www.google.com/base/feeds/snippets/16721...  healthy eating can be easy and convenient and ...   

                                                                     